In [1]:
import h2o

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 39 mins
H2O cluster timezone:,America/Santiago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,6 months and 29 days !!!
H2O cluster name:,H2O_from_python_maravenag_szc3fv
H2O cluster total nodes:,1
H2O cluster free memory:,2.205 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [27]:
train = h2o.upload_file("data/train.csv", destination_frame="train")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
train.describe()
#test no tiene la clases :o

Rows:891
Cols:12




,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,int,string,enum,real,int,int,int,real,enum,enum
mins,1.0,0.0,1.0,NaN,,0.42,0.0,0.0,693.0,0.0,,
mean,446.0,0.3838383838383838,2.3086419753086447,NaN,,29.69911764705884,0.5230078563411893,0.3815937149270483,260318.5491679275,32.20420796857465,,
maxs,891.0,1.0,3.0,NaN,,80.0,8.0,6.0,3101298.0,512.3292,,
sigma,257.3538420152301,0.4865924542648575,0.8360712409770491,NaN,,14.526497332334035,1.1027434322934315,0.8060572211299488,471609.26868834975,49.69342859718089,,
zeros,0,549,0,0,,0,608,678,0,15,,
missing,0,0,0,0,0,177,0,0,230,0,687,2
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,nan,7.25,,S
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1.0,0.0,nan,71.2833,C85,C
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,nan,7.925,,S


In [29]:
train = train.drop("PassengerId")
train = train.drop("Name")
train = train.drop("Cabin")
train = train.drop("Ticket")

In [30]:
train, test = train.split_frame(ratios=[0.7])

In [35]:
train.head()

Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,1,female,38,1,0,71.2833,C
1,3,female,26,0,0,7.925,S
0,3,male,35,0,0,8.05,S
0,1,male,54,0,0,51.8625,S
0,3,male,2,3,1,21.075,S
1,2,female,14,1,0,30.0708,C
1,3,female,4,1,1,16.7,S
1,1,female,58,0,0,26.55,S
0,3,male,20,0,0,8.05,S
0,3,male,39,1,5,31.275,S


In [36]:
train['Survived'] = train['Survived'].asfactor()
train['Pclass'] = train['Pclass'].asfactor()
train['Sex'] = train['Sex'].asfactor()
train['Embarked'] = train['Embarked'].asfactor()

test['Survived'] = test['Survived'].asfactor()
test['Pclass'] = test['Pclass'].asfactor()
test['Sex'] = test['Sex'].asfactor()
test['Embarked'] = test['Embarked'].asfactor()

#### Correr un autoML como baseline

In [37]:
from h2o.automl import H2OAutoML

In [38]:
target = "Survived"

In [39]:
train_cols = [x for x in train.col_names if x not in ['Survived']]

In [41]:
aml = H2OAutoML(max_runtime_secs = 100)
aml.train(x = train_cols, y = target,
          training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [42]:
lb = aml.leaderboard
print(lb)

model_id,auc,logloss
StackedEnsemble_BestOfFamily_0_AutoML_20181004_210221,0.864279,0.422145
StackedEnsemble_AllModels_0_AutoML_20181004_210221,0.863625,0.421457
GBM_grid_0_AutoML_20181004_210221_model_5,0.859924,0.434899
GBM_grid_0_AutoML_20181004_210221_model_7,0.856311,0.459063
GLM_grid_0_AutoML_20181004_210221_model_0,0.854106,0.447691
GBM_grid_0_AutoML_20181004_210221_model_1,0.85348,0.44485
GBM_grid_0_AutoML_20181004_210221_model_2,0.853119,0.445809
GBM_grid_0_AutoML_20181004_210221_model_6,0.847978,0.592098
XRT_0_AutoML_20181004_210221,0.847185,0.575999
GBM_grid_0_AutoML_20181004_210221_model_3,0.846393,0.457228


#### Ocupando un Autoencoder

In [51]:
from h2o.estimators import H2OAutoEncoderEstimator

In [82]:
autoencoder = H2OAutoEncoderEstimator(activation="Tanh",
                              hidden=[5],
                              ignore_const_cols=False,
                              epochs=1000)

In [83]:
autoencoder.train(x=train_cols, training_frame=train)

deeplearning Model Build progress: |██████████████████████████████████████| 100%


In [84]:
autoencoder._model_json["output"]["training_metrics"]


ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.005040743728558393
RMSE: 0.07099819524860046


In [93]:
train_deep_feats = autoencoder.deepfeatures(train, 0)
deep_feats = deep_feats.cbind(train['Survived'])
deep_cols = [x for x in deep_feats.col_names if x not in ['Survived']]

deepfeatures progress: |██████████████████████████████████████████████████| 100%


In [95]:
autoencoder.train(x=train_cols, training_frame=test)
autoencoder._model_json["output"]["training_metrics"]

deeplearning Model Build progress: |██████████████████████████████████████| 100%

ModelMetricsAutoEncoder: deeplearning
** Reported on train data. **

MSE: 0.006234375483659974
RMSE: 0.07895806154953385


In [103]:
test_deep_feats = autoencoder.deepfeatures(test, 0)
test_deep_feats = test_deep_feats.cbind(test['Survived'])

deepfeatures progress: |██████████████████████████████████████████████████| 100%


In [107]:
deep_feats.head()

DF.L1.C1,DF.L1.C2,DF.L1.C3,DF.L1.C4,DF.L1.C5,Survived
-0.273902,0.96165,0.910731,0.574897,-0.98633,1
-0.167885,0.610158,-0.630537,0.80459,0.871302,1
-0.827111,-0.932499,-0.637633,0.80221,0.87075,0
-0.86055,-0.400184,-0.39584,0.831914,-0.986425,0
-0.828361,-0.932076,-0.642454,0.80213,0.872174,0
-0.155802,0.63537,0.580681,-0.804745,-0.0508045,1
-0.170349,0.611169,-0.632936,0.804443,0.872034,1
-0.277788,0.96124,-0.385314,0.833995,-0.986361,1
-0.827417,-0.932452,-0.638025,0.801942,0.871072,0
-0.828341,-0.932167,-0.642105,0.802866,0.871221,0


In [108]:
test_deep_feats.head()

DF.L1.C1,DF.L1.C2,DF.L1.C3,DF.L1.C4,DF.L1.C5,Survived
0.925153,0.612716,-0.510325,-0.816533,0.931002,0
0.926149,0.748487,-0.212582,0.854018,-0.694501,1
0.767294,0.596371,0.698844,-0.902859,0.463372,0
0.926058,0.60685,-0.530862,-0.808017,-0.699674,1
0.772524,0.596726,0.691351,-0.906183,0.464983,0
-0.824052,0.47891,-0.55041,0.852127,0.930992,1
-0.822958,0.479146,-0.55038,0.853038,0.930967,0
0.926755,0.607375,-0.53598,-0.811229,-0.6981,0
0.92498,0.61279,-0.508033,-0.814611,0.93092,0
0.923436,-0.353502,0.61122,0.843367,0.537947,0


In [109]:
deep_feats_names = [x for x in deep_feats.col_names if x not in ["Survived"]]
deep_feats_names

['DF.L1.C1', 'DF.L1.C2', 'DF.L1.C3', 'DF.L1.C4', 'DF.L1.C5']

In [110]:
aml2 = H2OAutoML(max_runtime_secs = 100)
aml2.train(x = deep_feats_names, y = target,
          training_frame = deep_feats, validation_frame=test_deep_feats)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [111]:
lb = aml2.leaderboard
print(lb)

model_id,auc,logloss
StackedEnsemble_AllModels_0_AutoML_20181004_212202,0.860965,0.447042
StackedEnsemble_BestOfFamily_0_AutoML_20181004_212202,0.858162,0.448842
GBM_grid_0_AutoML_20181004_212202_model_14,0.857862,0.467077
GBM_grid_0_AutoML_20181004_212202_model_5,0.852709,0.448698
GBM_grid_0_AutoML_20181004_212202_model_3,0.848548,0.455586
GBM_grid_0_AutoML_20181004_212202_model_9,0.847645,0.521087
GBM_grid_0_AutoML_20181004_212202_model_2,0.842714,0.462415
GBM_grid_0_AutoML_20181004_212202_model_1,0.842188,0.460997
GBM_grid_0_AutoML_20181004_212202_model_6,0.841179,0.537509
XRT_0_AutoML_20181004_212202,0.838846,0.581463
